# --- Day 5: Print Queue ---
https://adventofcode.com/2024/day/5

In [2]:
def getRules():
    with open("rules.txt") as file:
        return file.read()

def getUpdates():
    with open("updates.txt") as file:
        return file.read()

In [3]:
from collections import defaultdict

# Formatting
rules = getRules()
rules = [[int(y) for y in x.split("|")] for x in rules.split("\n")]
updates = getUpdates()
updates = [[int(y) for y in x.split(",")] for x in updates.split("\n")]

ordering = defaultdict(list)
for rule in rules:
    ordering[rule[0]].append(rule[1])

validUpdates = []
for update in updates:
    # Keep track of progress through the current update
    progress = []
    valid = True

    # Loop through all values in the current update
    for val in update:
        # Check to see if current val is valid
        for i in ordering[val]:
            if i in progress:
                valid = False
        progress.append(val)
    if valid:
        validUpdates.append(progress)

validMiddlePageSum = 0
for update in validUpdates:
    validMiddlePageSum += update[len(update)//2]

print(f"Sum of middle page numbers in correctly-ordered updates: {validMiddlePageSum}")

Sum of middle page numbers in correctly-ordered updates: 5509


# Part 1 Different Approach
(In my opinion this is a better approach for clarity's sake)

In [5]:
# Formatting
rules = getRules()
rules = [[int(y) for y in x.split("|")] for x in rules.split("\n")]
updates = getUpdates()
updates = [[int(y) for y in x.split(",")] for x in updates.split("\n")]

# This function checks to see if an update is already sorted
# Credit to Zack for thinking of this cool approach
def checkSorted(update, rules) -> bool:
    # Loop through all rules
    for before, after in rules:
        # If a rule is violated, return false
        try:
            if update.index(before) > update.index(after):
                return False
        except ValueError:
            continue

    # If we make it through with no rule violations, return True
    return True

validMiddlePageSum = 0
for update in updates:
    if checkSorted(update, rules):
        validMiddlePageSum += update[len(update)//2]

print(f"Sum of middle page numbers in correctly-ordered updates: {validMiddlePageSum}")

Sum of middle page numbers in correctly-ordered updates: 5509


# --- Part Two ---

In [9]:
# Shoutout to Zack for the hints on this part
# Formatting
rules = getRules()
rules = [[int(y) for y in x.split("|")] for x in rules.split("\n")]
updates = getUpdates()
updates = [[int(y) for y in x.split(",")] for x in updates.split("\n")]

# Sort update list based on rules. Return new update list
def sortUpdate(update, rules) -> list[int]:
    sortedUpdate = update.copy()
    # Loop through all rules
    for before, after in rules:
        # If a rule is violated, swap the indexes
        try:
            beforeIndex = sortedUpdate.index(before)
            afterIndex = sortedUpdate.index(after)
            if beforeIndex > afterIndex:
                sortedUpdate[beforeIndex], sortedUpdate[afterIndex] = sortedUpdate[afterIndex], sortedUpdate[beforeIndex]
        except ValueError:
            continue

    # Return the sorted update
    return sortedUpdate

# Loop through all updates
invalidMiddlePageSum = 0
for update in updates:
    # If an update violates any rules
    if sortUpdate(update, rules) != update:

        # Sort the update and continue to sort until it no longer changes
        sortedUpdate = sortUpdate(update, rules)
        while sortUpdate(sortedUpdate, rules) != sortedUpdate:
            sortedUpdate = sortUpdate(sortedUpdate, rules)

        # Add the middle page to the sum of invalid update middle pages
        invalidMiddlePageSum += sortedUpdate[len(sortedUpdate)//2]

print(f"Sum of middle page numbers in incorrectly-ordered updates: {invalidMiddlePageSum}")

Sum of middle page numbers in incorrectly-ordered updates: 4407
